# Implement DEAP (Distributed Evolutionary Algorithms) using Python.

In [2]:
!pip install deap

  Obtaining dependency information for deap from https://files.pythonhosted.org/packages/e9/bd/654aa594a0ac14063fe591c3339062441eeda3815017e2f66d0b762cccb0/deap-1.4.2-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/109.8 kB ? eta -:--:--
   --- ------------------------------------ 10.2/109.8 kB ? eta -:--:--
   ------- ------------------------------- 20.5/109.8 kB 217.9 kB/s eta 0:00:01
   ------- ------------------------------- 20.5/109.8 kB 217.9 kB/s eta 0:00:01
   -------------- ------------------------ 41.0/109.8 kB 219.4 kB/s eta 0:00:01
   -------------- ------------------------ 41.0/109.8 kB 219.4 kB/s eta 0:00:01
   -------------- ------------------------ 41.0/109.8 kB 219.4 kB/s eta 0:00:01
   -------------- ------------------------ 41.0/109.8 kB 219.4 kB/s eta 0:00:01
   -------------- ------------------------ 41.0/109.8 kB 219.4 kB/s eta 0:00:01
   --------------------- ----------------- 61.4/109.8 kB 131.3 kB/s eta 0:00:01
   ------------

In [1]:
import random
from deap import creator, base, tools, algorithms

deap.creator: Used to define new types like Fitness and Individual.

deap.base: Contains the Toolbox where we register evolutionary operators.

deap.tools: Contains ready-made evolutionary operators like crossover, mutation, selection.

deap.algorithms: Contains some standard algorithms like varAnd

In [2]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

An Individual = a list of 100 numbers (0 or 1).

Fitness = how good the individual is. For us, that means: how many 1s are in the list?

In [3]:
toolbox = base.Toolbox()

toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=100)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

Toolbox: Think of toolbox as the factory where you register how to create individuals and apply evolutionary operators.

attr_bool: Creates a random 0 or 1. This is our gene generator.

individual: Creates an Individual with 100 random 0s or 1s. Uses initRepeat to repeat attr_bool 100 times.

population: Creates a population (a list of individuals).

In [4]:
def evalOneMax(individual):
    return sum(individual),

For the OneMax problem, we simply count how many 1s are in the individual. The , makes sure it returns a tuple (because DEAP expects fitness as a tuple).

In [5]:
toolbox.register("evaluate", evalOneMax)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

population = toolbox.population(n=300)

Evaluate by counting 1s.

Mate (combine two lists) using 2-point crossover.

Mutate (flip 0 to 1 or 1 to 0) with a small chance (5%).

Select using tournaments (pick the best out of 3 random lists).

We start with 300 random lists (each 100 numbers long).

In [8]:
NGEN=40
for gen in range(NGEN):
    offspring = algorithms.varAnd(population, toolbox, cxpb=0.5, mutpb=0.1)
    fits = toolbox.map(toolbox.evaluate, offspring)
    for fit, ind in zip(fits, offspring):
        ind.fitness.values = fit
    population = toolbox.select(offspring, k=len(population))
top10 = tools.selBest(population, k=10) # We get the top 10 best lists 
print(len(top10[0]))

100


We run for 40 generations (like 40 rounds).

50% chance for crossover (mixing lists),

10% chance for mutation (flipping bits).

Count the 1s in each child list.

Pick the best lists to be the new parents

In [9]:
print("Best fitness:", top10[0].fitness.values[0])

Best fitness: 99.0


Q: What is DEAP in Python?
A: DEAP (Distributed Evolutionary Algorithms in Python) is a library used for creating evolutionary algorithms like genetic algorithms (GA), genetic programming (GP), and evolution strategies.

Q: What is the objective of the code you provided?
A: The goal is to evolve a population of binary individuals to maximize the sum of 1s in the list — known as the OneMax problem.

Q: What is the OneMax problem?
A: It’s a classic optimization problem where the fitness of a binary string is the number of 1s it contains. The optimal solution is a string of all 1s.

Q: What will happen if we set mutation probability to 0?
A: The algorithm may converge prematurely and get stuck in sub-optimal solutions because mutation adds diversity.

Q: Why is tournament selection used instead of random selection?
A: It maintains a balance between selective pressure and diversity. It gives a better chance for fitter individuals to survive.